In [1]:
from MFEA_lib.model import Model_Offical
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *
from MFEA_lib.operators.Search import *
from MFEA_lib.tasks.Benchmark.Funcs import WCCI22_benchmark

In [2]:
class IndividualDE(Individual):
    def __init__(self, genes, dim= None) -> None:
        super().__init__(genes, dim)
        if genes is None:
            self.genes: np.ndarray = np.random.rand(dim)
        self.transfer =False

In [3]:
ls_benchmark = []
ls_IndClass = []
for i in range(1, 11, 1):
    t, ic = WCCI22_benchmark.get_complex_benchmark(i)
    ls_benchmark.append(t)
    ls_IndClass.append(IndividualDE)

name_benchmark = np.arange(len(ls_benchmark)) + 1

In [4]:
model = MultiBenchmark(
    ls_benchmark= ls_benchmark,
    name_benchmark= name_benchmark,
    ls_IndClass= ls_IndClass,
    model = Model_Offical
)
model.compile(
    crossover= SBX_Crossover(nc = 2),
    mutation = Polynomial_Mutation(nm = 5),
    selection= ElitismSelection(random_percent= 0.),
)
model.fit(
    nb_inds_each_task=100,
    nb_inds_min=30,
    nb_generations=1000,
    LSA = True,
    evaluate_initial_skillFactor= True
)
a = model.run(
    nb_run= 3,
    save_path= "./model_saved/__static__/WCCI22_complex/model_official/model_benchmark_id_"
)

Time: 00m 50.13s  100 % [====================>]  Pop_size: 1.86E+02  ,  Cost: 6.01E+02  6.01E+02  ,  hecking...Checking...DONE!
Saved
Time: 00m 50.72s  100 % [====================>]  Pop_size: 1.86E+02  ,  Cost: 7.00E+02  7.00E+02  ,  Checking...Checking...DONE!
Saved
Time: 00m 45.32s  100 % [====================>]  Pop_size: 1.86E+02  ,  Cost: 5.26E+03  4.93E+03  ,  Checking...Checking...DONE!
Saved
Time: 00m 28.02s  100 % [====================>]  Pop_size: 1.86E+02  ,  Cost: 1.30E+03  1.30E+03  ,  Checking...Checking...DONE!
Saved


d:\Vincent\Documents\Lab\Competition\official\MFEA_MSO\MFEA_lib\model\utils.py:70: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.array(result)


Time: 00m 40.43s  100 % [====================>]  Pop_size: 1.86E+02  ,  Cost: 1.52E+03  1.52E+03  ,  Checking...Checking...DONE!
Saved
Time: 00m 53.83s  100 % [====================>]  Pop_size: 1.86E+02  ,  Cost: 5.69E+03  6.94E+03  ,  Checking...Checking...DONE!
Saved
Time: 00m 50.63s  100 % [====================>]  Pop_size: 1.86E+02  ,  Cost: 2.77E+03  2.63E+03  ,  Checking...Checking...DONE!
Saved
Time: 00m 37.87s  100 % [====================>]  Pop_size: 1.86E+02  ,  Cost: 5.21E+02  5.21E+02  ,  Checking...Checking...DONE!
Saved
Time: 00m 34.24s  100 % [====================>]  Pop_size: 1.86E+02  ,  Cost: 8.49E+03  1.62E+03  ,  Checking...Checking...DONE!
Saved
Time: 00m 45.49s  100 % [====================>]  Pop_size: 1.86E+02  ,  Cost: 2.25E+03  6.99E+03  ,  Checking...Checking...DONE!
Saved


Write the result complex benchmark to text file
---------------

In [21]:
def write_res(res, path_out):
    with open(path_out, 'w') as fi:
        for i in res:
            fi.write(str(int(i[0])) + ", ")
            for j in range(1, len(i)-1):
                fi.write(str(i[j])+", ")
            fi.write(str(i[len(i)-1])+"\n")

len_out_complex = 100

for id in range(1, 11):

    model_mbm = loadModel(f"model_saved\__static__\WCCI22_complex\model_official\model_benchmark_id_{id}.mso", ls_tasks=WCCI22_benchmark.get_complex_benchmark(id))

    res = model_mbm.ls_model[0].history_cost[:len_out_complex]
    for i in range(1,model_mbm.nb_run):
        res = np.concatenate((res, np.array(model_mbm.ls_model[i].history_cost[:100])), 1)
    res.shape[0]
    num_evals = (np.arange(1, res.shape[0] + 1) * 1000 * len(model_mbm.tasks)).reshape(-1,1)
    out = np.concatenate((num_evals, res), 1)



    write_res(out, f"./result/MTO-complex/MTOSOO_P{id}.txt")